<a href="https://colab.research.google.com/github/robert-sturrock/clue_boardgame_solver/blob/main/Clue_Board_Game_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solving Clue

Aim is to create a tool that successfully performs most of the reasoning for you in Clue

I'm kind of trying to create an app in some ways. Needs to have a log, it needs persistence, and it needs to take inputs each round.

What are the best ways to build this? 

* Create an object that represents all the clue options (ie the detective pad). Can duplicate this by player
* Need a model of both: 1) what I think is in the envelope 2) what is in everyone elses hand. They have the same base structure.

In [2]:
#@title Set up code here{display-mode: "form"} 
import numpy as np
import matplotlib as m
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import copy

import re as re
import pandas as pd
import unicodedata 
import requests
import seaborn as sns
%matplotlib inline

## Create base clue object

Want to list out all the characters and have a 0,1 against them. Dictionary

We can start by just making things work with characters and then building up

For each player move we want to record: player, person, character, weapon. Do this as records (?). I'm going to have to record it anyway...

Will this actually be interesting? Yes. I'll be able to simplify calculations by inputting player order (so it can x out other people's cards)

Need two things. One that records what things aren't, and one that marks down what they are (ie has either x, y, z card)

And then it will cross reference with what we know about others (ie they have this card). 

Main thing we are recording is a list of what's in each player's hand

### Hand object

In [3]:
class Hand:

  def __init__(self):
    # add base features:
    self.characters = {'mustard':[None,[]], 
                       'plum':[None,[]], 
                       'green':[None,[]], 
                       'peacock':[None,[]], 
                       'scarlet':[None,[]], 
                       'white':[None,[]]
                       }
    self.weapons = {'knife':[None,[]],
                    'candlestick':[None,[]], 
                    'revolver':[None,[]], 
                    'rope':[None,[]], 
                    'lead pipe':[None,[]], 
                    'wrench':[None,[]]
                    }
    self.rooms = {'hall':[None,[]],
                  'lounge':[None,[]], 
                  'dining room':[None,[]], 
                  'kitchen':[None,[]],
                  'ballroom':[None,[]], 
                  'conservatory':[None,[]],
                  'billiard room':[None,[]],
                  'library':[None,[]],
                  'study':[None,[]],
                  }
    self.moves = []

  def initial_cards(self, initial_cards):
    # use this to initialize your hand (where you have full knowledge of cards)
    for group in [self.characters, self.weapons, self.rooms]:  
      for (key,val) in group.items():
        if key in initial_cards:
          group[key][0] = 1
        else:
          group[key][0] = 0

  def round(self, move):
    self.moves.append(move)

In [4]:
tmp = Hand()

In [5]:
tmp.characters

{'green': [None, []],
 'mustard': [None, []],
 'peacock': [None, []],
 'plum': [None, []],
 'scarlet': [None, []],
 'white': [None, []]}

In [6]:
initial_cards = ['Mustard','Plum','Wrench']
tmp.initial_cards(initial_cards)
[tmp.characters, tmp.weapons]

[{'green': [0, []],
  'mustard': [0, []],
  'peacock': [0, []],
  'plum': [0, []],
  'scarlet': [0, []],
  'white': [0, []]},
 {'candlestick': [0, []],
  'knife': [0, []],
  'lead pipe': [0, []],
  'revolver': [0, []],
  'rope': [0, []],
  'wrench': [0, []]}]

Now let's create a second class of object called "Clue_Game" which holds multiple hands. It will also hold the "move" method that runs eliminations and so on. 

Eventually I'll also want to add a history object as well that allows you roll back one move to undo. Or even stores the entire history.

Each player gets their own detective pad which only marks down what cards they have. I then have an "inference pad" which looks across 

### Clue_Game Object


In [177]:
class Clue_Game:

  def __init__(self, players, initial_cards):
    # always list yourself as first  **TO DO**: maybe use a specific name like "self" and then name opponents
    self.players = players
    self.moves = []
    self.hands = []
    self.partial_info_number = 0
    
    # last turn values
    self.hands_prev = []
    self.moves_prev = []
    self.partial_info_number_prev = []

    # initiate Hands:
    for player in self.players:
      hand = {'player':player, 'hand':Hand()}
      
      if player == 'self':
        hand['hand'].initial_cards(initial_cards)

      if player != 'self':
        for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
          for (key,val) in group.items():
            if key in initial_cards:
              group[key][0] = 0   
                   
      self.hands.append(hand)
  
  def players_involved(self, player, responder):
    if responder == None:
      return [p for p in self.players if p != player]
    else:
      start, end = [self.players.index(x) for x in [player, responder]]
      if start <= end:
          return self.players[start:end+1]
      else:
          return self.players[start:] + self.players[:end+1]
  
  def eliminate_non_response(self, player, person, weapon, room):
    # for a specific player eliminate a person and weapon
    for hand in self.hands:
        if hand['player'] == player:
          hand['hand'].characters[person][0] = 0
          hand['hand'].weapons[weapon][0] = 0
          hand['hand'].rooms[room][0] = 0
  
  def player_has_card(self, card, responder):
    # identify if we know that a responder (with hidden response) is holding one 
    # of the cards that they have been asked about.
    has_card = 0
    for hand in self.hands:
      if hand['player'] == responder:
        for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
          for (key,val) in group.items():
            if key in [card] and group[key][0] == 1:
              has_card += 1
    return has_card

  def record_response(self, player, person, weapon, room, responder=None, response=None):

    # record what the response is  if you are told
    if response not in [None,'hidden']:
      for hand in self.hands:
        if hand['player'] == responder:
          for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]: 
              for (key,val) in group.items():
                if key == response:
                  group[key][0] = 1
      
        # eliminate the option for everyone else
        if hand['player'] != responder:
          for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
            for (key,val) in group.items():
              if key == response:
                group[key][0] = 0
                group[key][1] = []

    # record the clues as to the response if you don't see it 
    # partial_info_number indicates what options are possibilities
    if response == 'hidden':
      has_card = 0
      for card in [person, weapon, room]:
        has_card += self.player_has_card(card, responder)
      # if player doesn't have any of the cards in their hand then we can record a partial info number
      if has_card == 0:
        for hand in self.hands:
          if hand['player'] == responder:
            for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
              for (key,val) in group.items():
                if key in [person, weapon, room] and group[key][0] not in [0,1]:
                  group[key][1].append(tmp.partial_info_number)

        
      # increment info number
      self.partial_info_number += 1

  def check_for_eliminations(self):
    last_item_found = None
    # look for cases where partial_info_numbers cancel out revealing true answer
    hand_checks = 1
    while hand_checks > 0:
      for hand in self.hands:
        # for each partial info number look at each hand and count up instances
        for number in range(self.partial_info_number):
          check_pin_count = 0 

          for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
            for (key,val) in group.items():
              if number in group[key][1]:
                check_pin_count += 1
        
          # if instances = 1 then indicate player has that card
          # *Note*: we don't remove partial info number for a 1, as there is
          # a possibility that player doesn't have the other card
          # (this will only happen once per partial_info_number)
          if check_pin_count == 1:
            for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
              for (key,val) in group.items():
                if number in group[key][1]:
                  group[key][0] = 1
                  group[key][1].remove(number)
                  # record card found and player it belongs to
                  last_item_found = key
                  player_item_found = hand['player']
                  hand_checks += 1

      # eliminate newly found card/item from other hands
      for hand in self.hands:         
        for group in [hand['hand'].characters, hand['hand'].weapons, hand['hand'].rooms]:  
          for (key,val) in group.items():
            if key == last_item_found and hand['player'] != player_item_found:
              group[key][0] = 0
              group[key][1] = []

      hand_checks -=1
 
  def move(self, player, person, weapon, room, responder=None, response=None):
    
    # back up previous hand states
    self.hands_prev = copy.deepcopy(self.hands)
    self.moves_prev = copy.deepcopy(self.moves)
    self.partial_info_number_prev = copy.copy(self.partial_info_number)

    # global  = list(self.moves)

    # record move
    rec = {'player':player, 'person':person, 'weapon':weapon, 'room':room,
           'responder':responder, 'response':response}
    self.moves.append(rec)

    # establish players involved
    players_involved = self.players_involved(player, responder)

    # eliminate options on players that didn't play
    no_match_players = list(set(players_involved) - set([player,responder]))
    for p in no_match_players:
      self.eliminate_non_response(p, person, weapon, room)

    # record information learned from the response or lack of
    self.record_response(player, person, weapon, room, responder, response)

    # look for possible eliminations
    self.check_for_eliminations()

  def hand_status(self):
    '''Get a dataframe of current hand information for all players'''
    # records 
    hand_records = []
    partial_info_records = []
    for hand in self.hands:
      player = hand['player']
      hc = pd.DataFrame(hand['hand'].characters)
      hw = pd.DataFrame(hand['hand'].weapons)
      hr = pd.DataFrame(hand['hand'].rooms)
      # store card knowledge and partial knowledge
      hand_col = pd.concat([hc,hw,hr],axis=1).loc[0]
      partial_info_col = pd.concat([hc,hw,hr],axis=1).loc[1]
      hand_col['player'] = player
      partial_info_col['player'] = player
      # record info for player
      hand_records.append(hand_col)
      partial_info_records.append(partial_info_col)
  
    return [pd.DataFrame(hand_records).set_index('player').transpose(),
            pd.DataFrame(partial_info_records).set_index('player').transpose()]

  def moves_hist(self):
    '''readable dataframe of moves'''
    return pd.DataFrame(self.moves)

  def remove_last_move(self):
    '''Use this to correct an incorrect move'''
    self.moves = copy.deepcopy(self.moves_prev)
    self.hands = copy.deepcopy(self.hands_prev)
    self.partial_info_number = copy.copy(self.partial_info_number_prev)
    


## Testing / Explaination

### Basics

Initial set up ensures that all my cards are known, and are also marked off of other people's hands

In [ ]:
tmp = Clue_Game(players = ['self','mom','dad','isabel'], initial_cards=['mustard','plum','wrench'])
tmp.hands[0]['hand'].characters

{'green': [0, []],
 'mustard': [1, []],
 'peacock': [0, []],
 'plum': [1, []],
 'scarlet': [0, []],
 'white': [0, []]}

In [ ]:
tmp.hands[1]['hand'].characters

{'green': [None, []],
 'mustard': [0, []],
 'peacock': [None, []],
 'plum': [0, []],
 'scarlet': [None, []],
 'white': [None, []]}

We can see pretty clearly that the "players involved" label is working / respects the ordering of the players

In [ ]:
tmp = Clue_Game(players = ['self','mom','dad','isabel'], initial_cards=['mustard','plum','wrench'])
print(tmp.players_involved(player='self',responder='dad'))
print(tmp.players_involved(player='dad', responder='mom'))

['self', 'mom', 'dad']
['dad', 'isabel', 'self', 'mom']


A 'move' action is where one player guesses

In [ ]:
# try to do a single move 
tmp.move('self','white','wrench','ballroom','isabel')

We can see that for the "no match players" ie those that said they didn't have any of the suggested cards we have replaced `"White":None` with `Wrench:0` and similar for weapons. 

In [ ]:
print(tmp.hands[1]['hand'].characters)
print(tmp.hands[2]['hand'].characters)
print(tmp.hands[1]['hand'].weapons)
print(tmp.hands[2]['hand'].weapons)

{'mustard': [0, []], 'plum': [0, []], 'green': [None, []], 'peacock': [None, []], 'scarlet': [None, []], 'white': [0, []]}
{'mustard': [0, []], 'plum': [0, []], 'green': [None, []], 'peacock': [None, []], 'scarlet': [None, []], 'white': [0, []]}
{'knife': [None, []], 'candlestick': [None, []], 'revolver': [None, []], 'rope': [None, []], 'lead pipe': [None, []], 'wrench': [0, []]}
{'knife': [None, []], 'candlestick': [None, []], 'revolver': [None, []], 'rope': [None, []], 'lead pipe': [None, []], 'wrench': [0, []]}


The next bit is storing partial information about players' hands. We want to mark down when someone answers, and we don't know what card it is, what cards it could possibly be.

What is the best way to do this? We could try a probability matrix but I'm not sure that would work. I could do it literally with 1,1,1 and 2,2,2 markings. But ruling things out could be tricky. 

How would it work? Let's say Mom said ('White','Pipe). Dad answers. From that we know Dad has either ('White','Pipe'). Therefore we want to rule it out from Dad's hand if:
* we see 'White' or 'Pipe' in someone else's hand
* Someone else asks 'White','Candlestick' and Dad doesn't have (eliminating Pipe)

These are both fairly simple cases. And the code method is basically once first element of a hand `[None,[]]` becomes not `None` we then eliminate the second element, and go searching for numbers that  are on their own. eg. if a hand had:
`[None,[1,2,7]` and became `[0,[1,2,7]` collapse it to `[0,[]]` and then scan through all the second elements that we're in the list: `[1,2,7]` and see if any of them have only one left. Repeat the process as long as numbers are found. 

Also this will have to iterate across player hands. ie every time we figure out a particular player has a certain card we need to eliminate we need to go through every player again to check combinations. 

In [ ]:
tmp = Clue_Game(players = ['self','mom','dad','isabel','alison'], initial_cards=['Mustard','Plum','Wrench'])

A move where we see the response

In [ ]:
tmp.move('self','white','knife','ballroom','isabel',response='white')
tmp.hands[3]['hand'].characters

{'green': [None, []],
 'mustard': [None, []],
 'peacock': [None, []],
 'plum': [None, []],
 'scarlet': [None, []],
 'white': [1, []]}

And we can see that 'White' is now eliminated for others, even those who didn't have a non-response  (below only 'White' is marked off, 'Knife' is still None)

In [ ]:
print(tmp.hands[4]['hand'].characters)
print(tmp.hands[4]['hand'].weapons)


{'mustard': [None, []], 'plum': [None, []], 'green': [None, []], 'peacock': [None, []], 'scarlet': [None, []], 'white': [0, []]}
{'knife': [None, []], 'candlestick': [None, []], 'revolver': [None, []], 'rope': [None, []], 'lead pipe': [None, []], 'wrench': [None, []]}


### The inference layer

Add in markings for all possibilities for a particular response in the second list

In [ ]:
tmp = Clue_Game(players = ['self','mom','dad','isabel','alison'], initial_cards=['Mustard','Plum','Wrench'])
tmp.move('mom','white','knife','ballroom','isabel',response='hidden')
tmp.hand_status()[1]

player,self,mom,dad,isabel,alison
mustard,[],[],[],[],[]
plum,[],[],[],[],[]
green,[],[],[],[],[]
peacock,[],[],[],[],[]
scarlet,[],[],[],[],[]
white,[],[],[],[0],[]
knife,[],[],[],[0],[]
candlestick,[],[],[],[],[]
revolver,[],[],[],[],[]
rope,[],[],[],[],[]


You can now see above that both 'White' and 'Knife' have a 0 next to them to indicate a possibility

In [ ]:
tmp.move('mom','plum','knife','hall','isabel',response='hidden')
tmp.hand_status()[1]

player,self,mom,dad,isabel,alison
mustard,[],[],[],[],[]
plum,[],[],[],[1],[]
green,[],[],[],[],[]
peacock,[],[],[],[],[]
scarlet,[],[],[],[],[]
white,[],[],[],[0],[]
knife,[],[],[],"[0, 1]",[]
candlestick,[],[],[],[],[]
revolver,[],[],[],[],[]
rope,[],[],[],[],[]


### Inference elimination

After each new piece of information go through the partial_info numbers and try to make eliminations

In the game below we have: 
* Isabel has knife
* Mom has ballroom
* Dad has white

In [ ]:
tmp = Clue_Game(players = ['self','mom','dad','isabel','alison'], initial_cards=['mustard','plum','wrench'])
tmp.move('mom','white','knife','ballroom','isabel',response='hidden')
tmp.move('self','white','knife','hall','dad',response='white')
tmp.move('self','white','knife','ballroom','mom',response='ballroom')

print("Isabel's hand now only has partial info on 'Knife' because dad has 'white' and mom has 'ballroom'. Info on the game:")
display(tmp.hand_status()[0])
display(tmp.hand_status()[1])


Isabel's hand now only has partial info on 'Knife' because dad has 'white' and mom has 'ballroom'. Info on the game:


player,self,mom,dad,isabel,alison
mustard,1,0,0,0,0
plum,1,0,0,0,0
green,0,NaN,NaN,NaN,NaN
peacock,0,NaN,NaN,NaN,NaN
scarlet,0,NaN,NaN,NaN,NaN
white,0,0,1,0,0
knife,0,0,0,1,0
candlestick,0,NaN,NaN,NaN,NaN
revolver,0,NaN,NaN,NaN,NaN
rope,0,NaN,NaN,NaN,NaN


player,self,mom,dad,isabel,alison
mustard,[],[],[],[],[]
plum,[],[],[],[],[]
green,[],[],[],[],[]
peacock,[],[],[],[],[]
scarlet,[],[],[],[],[]
white,[],[],[],[],[]
knife,[],[],[],[],[]
candlestick,[],[],[],[],[]
revolver,[],[],[],[],[]
rope,[],[],[],[],[]


So we can now see that her hand has 'Knife' crossed off

### Look at all moves that have happened

In [ ]:
tmp.moves

[{'person': 'white',
  'player': 'mom',
  'responder': 'isabel',
  'response': 'hidden',
  'room': 'ballroom',
  'weapon': 'knife'},
 {'person': 'white',
  'player': 'self',
  'responder': 'dad',
  'response': 'white',
  'room': 'hall',
  'weapon': 'knife'},
 {'person': 'white',
  'player': 'self',
  'responder': 'mom',
  'response': 'ballroom',
  'room': 'ballroom',
  'weapon': 'knife'}]

And for a more readable version

In [ ]:
tmp.moves_hist()

,player,person,weapon,room,responder,response
0,mom,white,knife,ballroom,isabel,hidden
1,self,white,knife,hall,dad,white
2,self,white,knife,ballroom,mom,ballroom


### Edge cases:

Now let's try a None result. It's still fairly interpretable. Eliminates it for everyone except the player

In [ ]:
tmp = Clue_Game(players = ['self','mom','dad','isabel','alison'], initial_cards=['mustard','plum','wrench'])
tmp.move('dad','scarlet','knife','ballroom',None,response=None)

tmp.hand_status()[0]

player,self,mom,dad,isabel,alison
mustard,1,0,0,0,0
plum,1,0,0,0,0
green,0,NaN,NaN,NaN,NaN
peacock,0,NaN,NaN,NaN,NaN
scarlet,0,0,NaN,0,0
white,0,NaN,NaN,NaN,NaN
knife,0,0,NaN,0,0
candlestick,0,NaN,NaN,NaN,NaN
revolver,0,NaN,NaN,NaN,NaN
rope,0,NaN,NaN,NaN,NaN


Let's now do a long string of moves and see what happens

In [ ]:
tmp = Clue_Game(players = ['self','mom','dad','isabel','alison'], initial_cards=['mustard','plum','wrench'])

tmp.move('mom','white','knife','hall','isabel',response='hidden')
display(tmp.hand_status()[0])
display(tmp.hand_status()[1])

tmp.move('self','white','lead pipe','ballroom','dad',response='white')
display(tmp.hand_status()[0])
display(tmp.hand_status()[1])

tmp.move('dad','white','candlestick','hall','isabel',response='hidden') # we know "White" therefore we know Candlestick
display(tmp.hand_status()[0])
display(tmp.hand_status()[1])

player,self,mom,dad,isabel,alison
mustard,1,0,0,0,0
plum,1,0,0,0,0
green,0,NaN,NaN,NaN,NaN
peacock,0,NaN,NaN,NaN,NaN
scarlet,0,NaN,NaN,NaN,NaN
white,0,NaN,0,NaN,NaN
knife,0,NaN,0,NaN,NaN
candlestick,0,NaN,NaN,NaN,NaN
revolver,0,NaN,NaN,NaN,NaN
rope,0,NaN,NaN,NaN,NaN


player,self,mom,dad,isabel,alison
mustard,[],[],[],[],[]
plum,[],[],[],[],[]
green,[],[],[],[],[]
peacock,[],[],[],[],[]
scarlet,[],[],[],[],[]
white,[],[],[],[0],[]
knife,[],[],[],[0],[]
candlestick,[],[],[],[],[]
revolver,[],[],[],[],[]
rope,[],[],[],[],[]


player,self,mom,dad,isabel,alison
mustard,1,0,0,0,0
plum,1,0,0,0,0
green,0,NaN,NaN,NaN,NaN
peacock,0,NaN,NaN,NaN,NaN
scarlet,0,NaN,NaN,NaN,NaN
white,0,0,1,0,0
knife,0,NaN,0,NaN,NaN
candlestick,0,NaN,NaN,NaN,NaN
revolver,0,NaN,NaN,NaN,NaN
rope,0,NaN,NaN,NaN,NaN


player,self,mom,dad,isabel,alison
mustard,[],[],[],[],[]
plum,[],[],[],[],[]
green,[],[],[],[],[]
peacock,[],[],[],[],[]
scarlet,[],[],[],[],[]
white,[],[],[],[],[]
knife,[],[],[],[0],[]
candlestick,[],[],[],[],[]
revolver,[],[],[],[],[]
rope,[],[],[],[],[]


player,self,mom,dad,isabel,alison
mustard,1,0,0,0,0
plum,1,0,0,0,0
green,0,NaN,NaN,NaN,NaN
peacock,0,NaN,NaN,NaN,NaN
scarlet,0,NaN,NaN,NaN,NaN
white,0,0,1,0,0
knife,0,NaN,0,NaN,NaN
candlestick,0,NaN,NaN,NaN,NaN
revolver,0,NaN,NaN,NaN,NaN
rope,0,NaN,NaN,NaN,NaN


player,self,mom,dad,isabel,alison
mustard,[],[],[],[],[]
plum,[],[],[],[],[]
green,[],[],[],[],[]
peacock,[],[],[],[],[]
scarlet,[],[],[],[],[]
white,[],[],[],[],[]
knife,[],[],[],[0],[]
candlestick,[],[],[],[1],[]
revolver,[],[],[],[],[]
rope,[],[],[],[],[]


### Edge case: player is holding a card that you know they have and provides a hidden response

This has now been addressed by adding in the condition that in order for us to add a partial info number the player must not already have one of their cards in their hand that we know about. 

Example: if dad says ('knife','hall') and mom responds we should only put in a partial info number if we don't already know that mom has 'knife' or 'hall'. If we already know mom has 'knife' then she may just be showing that card to dad, we can't get any extra information from it. 

In [178]:
tmp = Clue_Game(players = ['self','mom','dad','isabel','alison'], initial_cards=['mustard','plum','wrench'])
tmp.move('self','plum','knife','hall','dad','hall')

# check that we know that "dad" has the card hall
tmp.player_has_card('hall','dad')

1

## Test move rollback

At first we know nothing about Dad's hand except that he doesn't have Wrench. But then we learn that he doesn't have White or Knife.

In [ ]:
tmp = Clue_Game(players = ['self','mom','dad','isabel','alison'], initial_cards=['mustard','plum','wrench'])
display(tmp.hands[2]['hand'].weapons)
tmp.move('mom','white','knife','ballroom','alison',response='Hidden')
print('Current hand')
display(tmp.hands[2]['hand'].weapons)
display(tmp.partial_info_number)

{'candlestick': [None, []],
 'knife': [None, []],
 'lead pipe': [None, []],
 'revolver': [None, []],
 'rope': [None, []],
 'wrench': [0, []]}

Current hand


{'candlestick': [None, []],
 'knife': [0, []],
 'lead pipe': [None, []],
 'revolver': [None, []],
 'rope': [None, []],
 'wrench': [0, []]}

0

But now we can see his hand as it was the prior move

In [ ]:
tmp.hands_prev[1]['hand'].weapons

{'candlestick': [None, []],
 'knife': [None, []],
 'lead pipe': [None, []],
 'revolver': [None, []],
 'rope': [None, []],
 'wrench': [0, []]}

Now let's revert back to the original state

In [ ]:
tmp.remove_last_move()
tmp.hands[1]['hand'].weapons

{'candlestick': [None, []],
 'knife': [None, []],
 'lead pipe': [None, []],
 'revolver': [None, []],
 'rope': [None, []],
 'wrench': [0, []]}

Also need to check partial info numbers worked. Now back to 0

In [ ]:
tmp.partial_info_number

0

## Other features to add:
* A way to roll back an incorrect move [Done] 
* Add in rooms throughout code [Done]
* Output current matrix / df of information [Done]
* visualisation of what cards have been most asked about.
* Random simulation (ie each person just guesses completely at random / or with base logic (ie guess about things you don't have)

**Error handling**:
* If no valid input (eg. "Hidden" vs "hidden") throw an error

**Key game elements to add**
* Better handling for None results (needs to realise that player likely has at least one card if they don't immediately accuse)

## Test Game 1: Feb 13th 2020



In [180]:
tmp.hand_status()[0]

player,self,mom,dad,isabel,alison
mustard,1.0,0.0,0.0,0.0,0.0
plum,1.0,0.0,0.0,0.0,0.0
green,0.0,NaN,NaN,NaN,NaN
peacock,0.0,NaN,NaN,NaN,NaN
scarlet,0.0,NaN,NaN,NaN,NaN
white,0.0,NaN,NaN,NaN,NaN
knife,0.0,0.0,NaN,NaN,NaN
candlestick,0.0,NaN,NaN,NaN,NaN
revolver,0.0,NaN,NaN,NaN,NaN
rope,0.0,NaN,NaN,NaN,NaN


In [181]:
tmp = Clue_Game(players = ['self','dad','mom','isabel'], initial_cards=['white','peacock','green','conservatory'])

Game starts:

In [182]:
tmp.move(player='dad',person='scarlet',weapon='lead pipe',room='lounge',responder='mom',response='hidden')

In [183]:
tmp.move(player='mom',person='plum',weapon='knife',room='conservatory',responder='self',response='conservatory')

In [184]:
tmp.move(player='isabel',person='plum',weapon='knife',room='billiard room',responder='dad',response='hidden')

In [185]:
tmp.move(player='self',person='white',weapon='lead pipe',room='study',responder='dad',response='lead pipe')

In [186]:
tmp.move(player='dad',person='scarlet',weapon='lead pipe',room='conservatory',responder='self',response='conservatory')

In [187]:
tmp.move(player='mom',person='scarlet',weapon='revolver',room='lounge',responder='dad',response='hidden')

In [188]:
tmp.move(player='isabel',person='scarlet',weapon='revolver',room='ballroom',responder='dad',response='hidden')

In [189]:
tmp.move(player='self',person='scarlet',weapon='revolver',room='study',responder='dad',response='scarlet')

In [190]:
tmp.move(player='dad',person='mustard',weapon='candlestick',room='ballroom',responder='mom',response='hidden')

In [191]:
tmp.move(player='mom',person='plum',weapon='knife',room='conservatory',responder='self',response='conservatory')

In [192]:
tmp.move(player='isabel',person='white',weapon='revolver',room='hall',responder='self',response='white')

In [193]:
tmp.move(player='self',person='green',weapon='revolver',room='ballroom',responder='mom',response='ballroom')

In [194]:
tmp.hand_status()[0]

player,self,dad,mom,isabel
mustard,0.0,NaN,NaN,NaN
plum,0.0,NaN,NaN,0.0
green,1.0,0.0,0.0,0.0
peacock,1.0,0.0,0.0,0.0
scarlet,0.0,1.0,0.0,0.0
white,1.0,0.0,0.0,0.0
knife,0.0,NaN,NaN,0.0
candlestick,0.0,NaN,NaN,NaN
revolver,0.0,0.0,NaN,0.0
rope,0.0,NaN,NaN,NaN


In [195]:
tmp.hand_status()[1]

player,self,dad,mom,isabel
mustard,[],[],[4],[]
plum,[],[1],[],[]
green,[],[],[],[]
peacock,[],[],[],[]
scarlet,[],"[2, 3]",[],[]
white,[],[],[],[]
knife,[],[1],[],[]
candlestick,[],[],[4],[]
revolver,[],"[2, 3]",[],[]
rope,[],[],[],[]


In [196]:
tmp.player_has_card('ballroom','mom')

1

In [197]:
tmp.move(player='dad',person='mustard',weapon='rope',room='ballroom',responder='mom',response='hidden')

In [198]:
tmp.hand_status()[0]

player,self,dad,mom,isabel
mustard,0.0,NaN,NaN,NaN
plum,0.0,NaN,NaN,0.0
green,1.0,0.0,0.0,0.0
peacock,1.0,0.0,0.0,0.0
scarlet,0.0,1.0,0.0,0.0
white,1.0,0.0,0.0,0.0
knife,0.0,NaN,NaN,0.0
candlestick,0.0,NaN,NaN,NaN
revolver,0.0,0.0,NaN,0.0
rope,0.0,NaN,NaN,NaN


In [199]:
tmp.hand_status()[1]

player,self,dad,mom,isabel
mustard,[],[],[4],[]
plum,[],[1],[],[]
green,[],[],[],[]
peacock,[],[],[],[]
scarlet,[],"[2, 3]",[],[]
white,[],[],[],[]
knife,[],[1],[],[]
candlestick,[],[],[4],[]
revolver,[],"[2, 3]",[],[]
rope,[],[],[],[]


Error state: I think my code is now assuming that mom must have had one of mustard or rope but that's not true. Could have  show ballroom even though i know what it is.

Solution. We need to look at what cards that player has in their hand (that we know about). If the ask matches any one card we know about then don't record any information. 

**[Update: now corrected - see status readings above]**

In [ ]:
tmp.move(player='mom',person='plum',weapon='rope',room='dining room',responder='isabel',response='hidden')

In [ ]:
tmp.move(player='isabel',person='green',weapon='candlestick',room='dining room',responder='self',response='green')

In [ ]:
tmp.move(player='self',person='green',weapon='candlestick',room='dining room',responder='mom',response='candlestick')

In [ ]:
tmp.move(player='dad',person='peacock',weapon='wrench',room='ballroom',responder='mom',response='hidden')

In [ ]:
tmp.move(player='mom',person='plum',weapon='candlestick',room='dining room',responder=None,response=None)

In [ ]:
tmp.move(player='isabel',person='peacock',weapon='lead pipe',room='ballroom',responder='self',response='peacock')

In [ ]:
tmp.move(player='self',person='peacock',weapon='revolver',room='dining room',responder=None,response=None)

In [ ]:
tmp.moves_hist()

,player,person,weapon,room,responder,response
0,dad,scarlet,lead pipe,lounge,mom,hidden
1,mom,plum,knife,conservatory,self,conservatory
2,isabel,plum,knife,billiard room,dad,hidden
3,self,white,lead pipe,study,dad,lead pipe
4,dad,scarlet,lead pipe,conservatory,self,conservatory
5,mom,scarlet,revolver,lounge,dad,hidden
6,isabel,scarlet,revolver,ballroom,dad,hidden
7,self,scarlet,revolver,study,dad,scarlet
8,dad,mustard,candlestick,ballroom,mom,hidden
9,mom,plum,knife,conservatory,self,conservatory


In [ ]:
tmp.hand_status()[1]

player,self,dad,mom,isabel
mustard,[],[],"[4, 5]",[]
plum,[],[1],[],[]
green,[],[],[],[]
peacock,[],[],[],[]
scarlet,[],"[2, 3]",[],[]
white,[],[],[],[]
knife,[],[1],[],[]
candlestick,[],[],[4],[]
revolver,[],"[2, 3]",[],[]
rope,[],[],[5],[6]
